In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Verbindungsstring zur MS SQL-Datenbank
conn_str = (
    r'Driver={SQL Server};'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur MS SQL-Datenbank herstellen und Daten abrufen
connection = pyodbc.connect(conn_str)
query = "SELECT timestamp, o, c, h, l FROM stock_data ORDER BY timestamp"
data = pd.read_sql(query, connection)

# Daten vorbereiten
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['c']])

# Aufteilung der Daten in Trainings- und Testdaten
train_data = scaled_data[:-30]
test_data = scaled_data[-30:]

# Funktion zum Erstellen der Trainingsdaten für das LSTM-Modell
def create_lstm_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        X.append(dataset[i:i+look_back, 0])
        Y.append(dataset[i+look_back, 0])
    return np.array(X), np.array(Y)

# Trainingsdaten erstellen
look_back = 30
train_X, train_Y = create_lstm_dataset(train_data, look_back)

# Daten für das LSTM-Modell reshape
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# LSTM-Modell erstellen
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modell trainieren
model.fit(train_X, train_Y, epochs=10, batch_size=32)

# Vorhersage für den nächsten Monat
test_X = test_data
test_X = np.reshape(test_X, (1, look_back, 1))
next_month_predictions = model.predict(test_X)

# Denormalisierung der Daten
next_month_predictions = scaler.inverse_transform(next_month_predictions)

# Index für die Vorhersage erstellen
predictions_index = pd.date_range(start=data['timestamp'].iloc[-31], periods=30, freq='D')

# DataFrame für die Vorhersagen erstellen
predictions_df = pd.DataFrame(next_month_predictions, index=predictions_index, columns=['Prediction'])

# Plot der historischen Daten und Vorhersagen
plt.plot(data['timestamp'], data['c'], label='Historical Data')
plt.plot(predictions_df.index, predictions_df['Prediction'], label='Next Month Predictions')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.show()


C:\Users\HerbyHerb\AppData\Local\Temp\ipykernel_12424\417937183.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


Epoch 1/10
10/10 [==============================] - 7s 25ms/step - loss: 0.0793
Epoch 2/10
10/10 [==============================] - 0s 22ms/step - loss: 0.0189
Epoch 3/10
10/10 [==============================] - 0s 21ms/step - loss: 0.0163
Epoch 4/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0136
Epoch 5/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0123
Epoch 6/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0112
Epoch 7/10
10/10 [==============================] - 0s 21ms/step - loss: 0.0101
Epoch 8/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0093
Epoch 9/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0086
Epoch 10/10
1/1 [==============================] - 1s 845ms/step


ValueError: Shape of passed values is (1, 1), indices imply (30, 1)